### Filtrando i dati

devo leggere tutt .csv e salvare solamente la 'close' di ogni csv

In [1]:
import cudf as cf
from bollinger_filter import backtest_bollinger_strategy

eurgbp = cf.read_csv('/home/edoardocame/Desktop/python_dir/data/eurgbp-m1-bid-2015-01-01-2025-01-30T14:40.csv')
train_size = 0.8
eurgbp_train = eurgbp.iloc[:int(len(eurgbp)*train_size)] #selects the first slice of data as training set
eurgbp_test = eurgbp.iloc[int(len(eurgbp)*train_size):] #$selects the second slice of data as test set


#audnzd = cf.read_csv('/home/edoardocame/Desktop/python_dir/data/audnzd-m1-bid-2024-01-01-2025-01-28.csv')
#eurchf = cf.read_csv('/home/edoardocame/Desktop/python_dir/data/eurchf-m1-bid-2024-01-01-2025-01-28.csv')

In [2]:
len(eurgbp['close'])

3755716

### Ottimizzazione dei parametri con pandas/polars
Devo ottimizzare i parametri dei backtest per poi testarli nel portafoglio combinato

In [3]:
#Questa è la via pandas, susseguentemente farò quella con polars;
#ci mettono lo stesso identico tempo in termini di velocità.

import pandas as pd
import time as t

my_combinations = pd.read_csv('/home/edoardocame/Desktop/python_dir/PythonMiniTutorials/trading strategies/MyOwnBacktester/transition to cuDF/my_combinations.csv', index_col=0)

start_time = t.time()

####################
my_combinations['result'] = 0
def apply_strategy(row):
    result = backtest_bollinger_strategy(data=eurgbp, lookback=row['lookback'], 
                                         sdev=row['sdev'], leverage=10, filter=row['filter'], fee_percentage=0.01)
    print(f"Processed lookback: {row['lookback']}, sdev: {row['sdev']}, result: {result}, filter: {row['filter']}")
    return result

my_combinations['result'] = my_combinations.apply(apply_strategy, axis=1)
####################

end_time = t.time() - start_time

print(f"Time elapsed: {end_time}")

Processed lookback: 30, sdev: 3, result: 7.436800927064959e-19, filter: 0
Processed lookback: 30, sdev: 3, result: 6.360538931016621e-12, filter: 1
Processed lookback: 30, sdev: 3, result: 8.156742769476181e-08, filter: 2
Processed lookback: 32, sdev: 3, result: 4.40598624927333e-19, filter: 0
Processed lookback: 32, sdev: 3, result: 3.761697557062713e-12, filter: 1
Processed lookback: 32, sdev: 3, result: 7.41659730810199e-08, filter: 2
Processed lookback: 34, sdev: 3, result: 2.6330704782794116e-19, filter: 0
Processed lookback: 34, sdev: 3, result: 1.961270331574602e-12, filter: 1
Processed lookback: 34, sdev: 3, result: 8.617462807155271e-08, filter: 2
Processed lookback: 36, sdev: 3, result: 1.7054393280148111e-19, filter: 0
Processed lookback: 36, sdev: 3, result: 1.1099738919239301e-12, filter: 1
Processed lookback: 36, sdev: 3, result: 9.050161496588406e-08, filter: 2
Processed lookback: 38, sdev: 3, result: 1.6286784480053385e-19, filter: 0
Processed lookback: 38, sdev: 3, res

In [6]:
my_combinations.to_csv('/home/edoardocame/Desktop/python_dir/PythonMiniTutorials/trading strategies/MyOwnBacktester/transition to cuDF/my_combinations_results.csv')

In [ ]:
#Ora con Polars:

#import polars as pl
#import time as t
#
#my_combinations = pl.read_csv('/home/edoardocame/Desktop/python_dir/transition to cuDF/my_combinations.csv')
#
#start_time = t.time()
#results = []
#for row in my_combinations.iter_rows(named=True): #named = True serve per indicare le colonne
#    result = backtest_bollinger_strategy(data=eurgbp, lookback=row['lookback'], sdev=row['sdev'])
#    print(f"Processed lookback: {row['lookback']}, sdev: {row['sdev']}, result: {result}")
#    results.append(result)
#
#my_combinations = my_combinations.with_columns(pl.Series(name="result", values=results))
#end_time = t.time() - start_time
#
#
#print(f"Time elapsed: {end_time}")

### Analisi dei migliori parametri
Qui analizzo quali sono i migliori parametri trovati per EUR/GBP;
Poi farò lo stesso con le altre coppie.

In [ ]:
import cudf as cf

my_combinations = cf.read_csv('/home/edoardocame/Desktop/python_dir/transition to cuDF/my_combinations_results_leverage.csv', index_col=0)
params = my_combinations[my_combinations['result'] == my_combinations['result'].max()]
print(params)

In [ ]:
from bollinger import backtest_bollinger_strategy
import cudf as cf

esempio = backtest_bollinger_strategy(data = eurgbp_test, lookback= 60, sdev= 6, 
                                      fee_percentage=0.0001, return_series=True, leverage=10)


import matplotlib.pyplot as plt
esempio.dropna(inplace=True)
plt.plot(esempio)
plt.title('Test set')
plt.show()

Qui voglio vedere se i filtri hanno avuto effetto;

In [2]:
import cudf as cf

df = cf.read_csv('/home/edoardocame/Desktop/python_dir/PythonMiniTutorials/trading strategies/MyOwnBacktester/transition to cuDF/my_combinations_results.csv')
df = df.dropna()

#Voglio vedere se la media dei risultati varia significativamente in base al filtro;
#Se non varia, posso escluderlo dalla strategia.
df.groupby('filter')['result'].mean()

filter
0    143.004716
2    151.564974
1     66.569068
Name: result, dtype: float64

#### Plot 3d dei risultati
Esempio di ottimizzazione con eur/gbp: mostriamo un grafico 3d dei risultati;

In [ ]:
import pandas as pd
import plotly.graph_objects as go

my_combinations = pd.read_csv('/home/edoardocame/Desktop/python_dir/transition to cuDF/my_combinations_results_leverage.csv')

#Subset the data keeping only sdevs between 4 and 10
my_combinations = my_combinations[(my_combinations['sdev'] > 3) & (my_combinations['sdev'] < 10)]


z = my_combinations['result'].values
x = my_combinations['sdev'].values
y = my_combinations['lookback'].values

fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(size=12, color=z, colorscale='Viridis', opacity=0.8))])
fig.update_layout(scene = dict(xaxis_title='Standard Deviation', yaxis_title='Lookback', zaxis_title='Result'))
fig.show()

